# Icosa Computing Combinatorial Reasoning Demo

## Set Up

We have uploaded the package to PyPi, so the combinatorial reasoning pipeline can be accessed very easily via a pip install.

In [3]:
%pip install langchain_icosa==1.0.1 # download package if not already installed

  Attempting uninstall: langchain_icosa
    Found existing installation: langchain_icosa 1.0.0
    Uninstalling langchain_icosa-1.0.0:
      Successfully uninstalled langchain_icosa-1.0.0

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# from langchain_icosa.combinatorial_reasoning import CombinatorialReasoningLLM, CombinatorialReasoningCallbackHandler
from langchain_icosa.combinatorial_reasoning import CombinatorialReasoningLLM, CombinatorialReasoningCallbackHandler
from langchain_openai import ChatOpenAI # this is our zero-shot comparison
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from getpass import getpass
from IPython.display import display, Markdown

In [7]:
API_KEY = getpass('Enter OpenAI API key')

## Comparison of Zero-Shot and CR

We will use `GPT-3.5-Turbo` for the base LLM. 

The actual compute of the Combinatorial Reasoning pipeline takes place off-premises on Icosa's cloud servers, which enables the library to be installed with fewer external dependencies and performance requirements.

In [8]:
ZeroShotLLM = ChatOpenAI(openai_api_key=API_KEY, model = 'gpt-3.5-turbo')
CrLLM = CombinatorialReasoningLLM(openai_api_key=API_KEY, model = 'gpt-3.5-turbo')
callback = CombinatorialReasoningCallbackHandler() # callback to hook into the reason selection process

The Combinatorial Reasoning LLM supports user-adjustable hyperparameters. However, we have already tuned these hyperparameters, and so for almost all use cases, the default parameters are sufficient.

In [9]:
display(CrLLM.dict())

{'model_name': 'CombinatorialReasoningLLM',
 'linear_sensitivity': 3.5341454578705958,
 'thresh_param': 2.4601753808001217,
 'risk_param': 0.38900003710737635,
 'weight': 2,
 'model': 'gpt-3.5-turbo',
 '_type': 'CombinatorialReasoningLLM'}

The Combinatorial Reasoning LLM supports answers with and without reasoning, adjustable via the `responseType` parameter in the invoke method. This method also supports overloading the hyperparameters. For the sake of demonstration, we include the reasoning in all responses.

In [10]:
def compare_llms(prompt, seed = 0): 
    display(Markdown(f"### Prompt\n{prompt}"))
    zero_shot_solution = ZeroShotLLM.invoke(prompt).content
    display(Markdown("### Zero-Shot Solution"))
    display(Markdown(zero_shot_solution))
    cr_solution = CrLLM.invoke(prompt, responseType='answerWithReasoning', seed = seed, config={'callbacks': [callback]})
    display(Markdown("### CR Solution"))
    display(Markdown(cr_solution))
    display(Markdown("### CR Statistics"))
    formatted_stats = (
        f"- **Number of Distinct Reasons**: {callback.stats[0]['num_distinct_reasons']}\n" +
        f"- **Number of Selected Reasons**: {callback.stats[0]['num_selected_reasons']}\n" + 
        f"- **Percentage Selected**: {callback.stats[0]['proportion_selected']:.2%}"
    )
    display(Markdown(formatted_stats))
    

In [ ]:
def show_reasons():
    display(Markdown("### Final Selected Reasons"))
    selected_reasons = callback.data['selected_reasons'][0]
    formmated_reasons = "\n- ".join([f"{reason[0]}" for reason in selected_reasons])
    display(Markdown("- " + formmated_reasons))


### Animal Prompt

In [14]:
animal_prompt = "There are six animals: lion, hyena, elephant, deer, cat and mouse. Separate them to three spaces to minimize conflict."
compare_llms(animal_prompt)

### Prompt
There are six animals: lion, hyena, elephant, deer, cat and mouse. Separate them to three spaces to minimize conflict.

### Zero-Shot Solution

1. Lion, hyena, cat
2. Elephant, deer
3. Mouse

### CR Solution

The W-Values indicate that the most reliable information comes from the statements with a W-Value of 0.103. 

From these statements, we can gather the following information:
1. Lion, hyena, and elephant should be placed together to minimize conflicts among them.
2. Lion and hyena should be kept separate to minimize conflict.
3. Elephant should be isolated in its own space due to its size and potential impact on other animals.
4. Group herbivores together to decrease the chance of conflict.
5. Group elephant and deer together as herbivores who are less likely to compete with the carnivores.

Considering all the information provided, the best way to separate the animals into three spaces would be:
Space 1: Lion, hyena, elephant
Space 2: Deer, cat
Space 3: Mouse

SOLUTION: (lion, hyena, elephant) - (deer, cat) - (mouse)

### CR Statistics

- **Number of Distinct Reasons**: 153
- **Number of Selected Reasons**: 13
- **Percentage Selected**: 8.50%

## River Crossing Prompt

In [ ]:
river_prompt = '''
There is a man, a sheep and a boat with space for one human and one animal on one side of a river. How do the man and sheep get to the other side of the river?
'''
compare_llms(river_prompt)

## Trolley Prompt

In [ ]:
trolley_prompt = '''
"Imagine a runaway trolley is hurtling down a track towards five dead people. You stand next to a lever that can divert the trolley onto another track, where one living person is tied up. Do you pull the lever?"
'''
compare_llms(trolley_prompt)

## Usage with LangChain

In [ ]:
prompt = PromptTemplate.from_template("Should I buy {ticker} stock today?")
model = CombinatorialReasoningLLM(openai_api_key=API_KEY)
chain = prompt | model | StrOutputParser()

chain.invoke({'ticker': 'AAPL'})